In [2]:
import pandas as pd
# from sqlalchemy import create_engine

# Step 1: Load your CSV into a DataFrame
csv_path = r"C:\Users\User\Documents\GitHub\credit-risk-sentiment-nps\data\raw\credit-risk-sentiment-nps-sample-orig.csv"
df = pd.read_csv(csv_path)
df = df.drop_duplicates(subset='CLIENTNUM')


In [3]:
# populate the 'Attrition_Type' column  with alternating values "Voluntary" and "Involuntary" row by row:
import pandas as pd

# Get indices where Attrition_Flag is 'Attrited Customer'
attrited_indices = df[df['Attrition_Flag'] == 'Attrited Customer'].index

# Generate alternating values
# to make 
# 📊 Estimated Involuntary Attrition Rate (U.S. Market)
# Typical Range: 2% to 5% annually
# Higher for subprime customers: Up to 10% or more
# Lower for premium cardholders or strong credit customers: Often <2%

alternating_values = ['Voluntary' if i % 4 == 1 else 'Involuntary' for i in range(len(attrited_indices))]
# alternating_values = ['Voluntary' if i % 2 == 1 else 'Involuntary' for i in range(len(attrited_indices))]

# Create the Attrition_Type column with NaN first
df['Attrition_Type'] = pd.NA

# Assign alternating values only to 'Attrited Customer' rows
df.loc[attrited_indices, 'Attrition_Type'] = alternating_values

df['Attrition_Type'] = df['Attrition_Type'].fillna('Existing Customer')


In [4]:
# Attrited Customer only
df_attr=  df[df['Attrition_Flag'] == 'Attrited Customer']
df_attr.to_csv( r"C:\Users\User\Documents\GitHub\credit-risk-sentiment-nps\data\raw\attrition_sample.csv",index = False)

csv_path2 = r"C:\Users\User\Documents\GitHub\credit-risk-sentiment-nps\data\raw\attrited_only_voluntary_personal_sentiment_nps_sample.csv"
volunary = pd.read_csv(csv_path2, encoding='latin1')

csv_path3 = r"C:\Users\User\Documents\GitHub\credit-risk-sentiment-nps\data\raw\attrited_only_involuntary_business_sentiment_nps_sample.csv"
business = pd.read_csv(csv_path3, encoding='latin1')


In [5]:
# df['Income_Category'].unique()

In [6]:
import numpy as np
import pandas as pd

# Define income ranges for each category
income_ranges = {
    'Less than $40K': (20000, 39999),
    '$40K - $60K': (40000, 59999),
    '$60K - $80K': (60000, 79999),
    '$80K - $120K': (80000, 119999),
    '$120K +': (120000, 200000),  # or any reasonable upper bound
}

# Apply function to assign random incomes
def assign_random_income(row):
    category = row['Income_Category']
    if category in income_ranges:
        low, high = income_ranges[category]
        return np.random.randint(low, high + 1)
    else:
        return np.nan  # keep NaN for 'Unknown' or others

df['annual_income'] = df.apply(assign_random_income, axis=1)
df['monthly_income'] = df['annual_income'] /12
df['DTI_ratio'] = df['Total_Revolving_Bal']/df['monthly_income']*100


# to get Risk_Level
def classify_dti_risk(dti):
    if dti < 36:
        return 'Low Risk'
    elif 36 <= dti <= 49:
        return 'Medium Risk'
    else:
        return 'High Risk'

df['DTI_Risk_Level'] = df['DTI_ratio'].apply(classify_dti_risk)


In [7]:
import numpy as np

# Set seed for reproducibility (optional)
np.random.seed(42)

# Assign random credit scores between 300 and 850 to 'Existing Customer' rows
df.loc[df['Attrition_Flag'] == 'Existing Customer', 'Credit_Score'] = np.random.randint(
    450, 851, size=(df['Attrition_Flag'] == 'Existing Customer').sum()
)

# Assign random credit scores between 300 and 850 to df['Attrition_Type']'=='Voluntary'
df.loc[df['Attrition_Type'] == 'Voluntary', 'Credit_Score'] = np.random.randint(
    450, 851, size=(df['Attrition_Type'] == 'Voluntary').sum()
)


# Assign random credit scores between 300 and 850 to df['Attrition_Type']'=='Involuntary'
df.loc[df['Attrition_Type'] == 'Involuntary', 'Credit_Score'] = np.random.randint(
    450, 701, size=(df['Attrition_Type'] == 'Involuntary').sum()
)


In [8]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)

# Populate  'Loan_Default_History' with binary flags: 1 = Defaulted, 0 = No Default
df['Loan_Default_Flag'] = np.where(
    df['Attrition_Flag'] == 'Attrited Customer',
    np.random.choice([1, 0], size=len(df), p=[0.2, 0.8]),
    np.random.choice([1, 0], size=len(df), p=[0.1, 0.9])
)

df['Loan_Default_Flag'] = np.where(
    df['Attrition_Type'] == 'Involuntary',
    np.random.choice([1, 0], size=len(df), p=[0.4, 0.6]),
    np.random.choice([1, 0], size=len(df), p=[0.2, 0.8])
)


# Initialize empty columns
df['Number_of_Defaults'] = 0
df['Last_Default_Date'] = pd.NaT
df['Default_Amount'] = 0.0

# Get indices where customer defaulted
defaulted_idx = df[df['Loan_Default_Flag'] == 1].index

# Generate synthetic data for defaulted customers
df.loc[defaulted_idx, 'Number_of_Defaults'] = np.random.randint(1, 6, size=len(defaulted_idx))

# Generate random dates in the past 5 years
def random_date_within_last_n_years(n):
    days_ago = np.random.randint(30, 365 * n)
    return datetime.today() - timedelta(days=int(days_ago))

df.loc[defaulted_idx, 'Last_Default_Date'] = [random_date_within_last_n_years(5) for _ in range(len(defaulted_idx))]

# Generate random default amounts between $500 and $20,000
df.loc[defaulted_idx, 'Default_Amount'] = np.round(np.random.uniform(500, 20000, size=len(defaulted_idx)), 2)


In [9]:
df.head(1)

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Attrition_Type,annual_income,monthly_income,DTI_ratio,DTI_Risk_Level,Credit_Score,Loan_Default_Flag,Number_of_Defaults,Last_Default_Date,Default_Amount
0,A900000000,Attrited Customer,52,M,5,Undergraduate,Single,$60K - $80K,Gold,41,...,Involuntary,73432.0,6119.333333,41.131932,Medium Risk,477.0,1,1,2024-11-17 20:42:57.584306,8378.7


In [10]:
df.to_csv( r"C:\Users\User\Documents\GitHub\credit-risk-sentiment-nps\data\processed\processed_data.csv",index = False)

In [11]:
import numpy as np

def assign_bankruptcy(row):
    if row['Attrition_Flag'] == 'Attrited Customer':
        if row['Attrition_Type'] == 'Voluntary':
            return np.random.choice(['Yes', 'No'], p=[0.2, 0.8])
        elif row['Attrition_Type'] == 'Involuntary':
            return np.random.choice(['Yes', 'No'], p=[0.4, 0.6])
        else:
            return 'No'
    elif row['Attrition_Flag'] == 'Existing Customer':
        return np.random.choice(['Yes', 'No'], p=[0.05, 0.95])
    else:
        return 'No'

df['Bankruptcy_History'] = df.apply(assign_bankruptcy, axis=1)


In [12]:
# Example data
import pandas as pd

# Create bankruptcy flag: 1 if history is 'Yes', else 0
df['Bankruptcy_Flag'] = df['Bankruptcy_History'].apply(lambda x: 1 if x.lower() == 'yes' else 0)

# print(df)

def adjust_score(row):
    if row['Bankruptcy_Flag'] == 1:
        # Example: deduct 150 points if bankruptcy exists
        return max(row['Credit_Score'] - 150, 300)  # Ensure minimum score 300
    else:
        return row['Credit_Score']

df['Adjusted_Credit_Score'] = df.apply(adjust_score, axis=1)

# print(df)

def credit_score_to_moodys(score):
    if score >= 800:
        return 'Aaa'
    elif score >= 750:
        return 'Aa'
    elif score >= 700:
        return 'A'
    elif score >= 650:
        return 'Baa'
    elif score >= 600:
        return 'Ba'
    elif score >= 550:
        return 'B'
    elif score >= 500:
        return 'Caa'
    elif score >= 400:
        return 'Ca'
    else:
        return 'C'

df['Moodys_Rating'] = df['Adjusted_Credit_Score'].apply(credit_score_to_moodys)

# print(df)


In [13]:
# Part 1: Python Code — Credit Risk Scoring Pipeline with Bankruptcy

In [14]:
# # Step 5: Payment history impact
import numpy as np

# Define a mask to filter only relevant rows (not bankrupt)
non_bankrupt_mask = ~df['Bankruptcy_History'].str.lower().eq('yes')

# Step 1: Start by filling with 'good' for all non-bankrupt customers
df.loc[non_bankrupt_mask, 'Payment_History_Flag'] = 'good'

# Rule 1: Bankruptcy => bad payment history
df.loc[df['Bankruptcy_History'].str.lower() == 'yes', 'Payment_History_Flag'] = 'bad'

# Rule 2: Involuntary attrition but not bankrupt => 35% bad, 65% good
mask_involuntary = (df['Attrition_Type'] == 'Involuntary') & (df['Bankruptcy_History'].str.lower() != 'yes')
df.loc[mask_involuntary, 'Payment_History_Flag'] = np.where(
    np.random.rand(mask_involuntary.sum()) < 0.35, 'bad', 'good'
)


# I need to rethink about it
# Step 5: Payment history impact
df['Payment_Risk'] = df['Payment_History_Flag'].apply(lambda x: 'good' if x == 'good' else 'bad')
df.head(1)

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Loan_Default_Flag,Number_of_Defaults,Last_Default_Date,Default_Amount,Bankruptcy_History,Bankruptcy_Flag,Adjusted_Credit_Score,Moodys_Rating,Payment_History_Flag,Payment_Risk
0,A900000000,Attrited Customer,52,M,5,Undergraduate,Single,$60K - $80K,Gold,41,...,1,1,2024-11-17 20:42:57.584306,8378.7,Yes,1,327.0,C,bad,bad


In [18]:
df['estimated_annual_revenue'] = df['Total_Revolving_Bal'] * 0.20 + 50


In [21]:
df.to_csv( r"C:\Users\User\Documents\GitHub\credit-risk-sentiment-nps\data\processed\processed_data_ready_to_analyze.csv",index = False)
df.to_csv( r"C:\Users\User\Documents\GitHub\credit-risk-sentiment-nps\data\official\processed_data_ready_to_analyze.csv",index = False)

In [20]:
df.describe()

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,...,monthly_income,DTI_ratio,Credit_Score,Loan_Default_Flag,Number_of_Defaults,Last_Default_Date,Default_Amount,Bankruptcy_Flag,Adjusted_Credit_Score,estimated_annual_revenue
count,10128.000000,10128.000000,10128.000000,10128.000000,10128.000000,10128.000000,10128.000000,10128.000000,10128.000000,10127.000000,...,9016.000000,9016.000000,10128.000000,10128.000000,10128.000000,2255,10128.000000,10128.000000,10128.000000,10128.000000
mean,46.326521,2.346465,35.928910,3.812599,2.342121,2.455075,8634.557178,1162.947769,7471.577814,0.759941,...,5211.288164,30.768135,640.672788,0.222650,0.677626,2023-01-23 02:02:14.389859072,2275.305247,0.100316,625.625395,282.589554
min,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,...,1666.750000,0.000000,450.000000,0.000000,0.000000,2020-08-09 22:37:53.636954,0.000000,0.000000,301.000000,50.000000
25%,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2555.000000,360.000000,1324.750000,0.631000,...,2738.062500,6.177866,544.000000,0.000000,0.000000,2021-10-24 22:37:53.628002560,0.000000,0.000000,525.000000,122.000000
50%,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4549.000000,1276.500000,3474.500000,0.736000,...,4200.583333,24.654816,635.000000,0.000000,0.000000,2023-01-27 22:37:53.635172096,0.000000,0.000000,622.000000,305.300000
75%,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,11070.250000,1784.000000,9861.750000,0.859000,...,6674.645833,46.680241,737.000000,0.000000,0.000000,2024-04-23 22:37:53.632361984,0.000000,0.000000,730.000000,406.800000
max,73.000000,5.000000,56.000000,6.000000,12.000000,6.000000,35000.000000,2517.000000,34516.000000,3.397000,...,16664.583333,149.985103,850.000000,1.000000,5.000000,2025-07-06 22:37:53.642402,19999.460000,1.000000,850.000000,553.400000
std,8.016616,1.299112,7.986181,1.554332,1.015120,1.106440,9092.103865,815.058178,9093.547561,0.219207,...,3275.029033,29.151431,114.324010,0.416046,1.432613,NaN,5008.536585,0.300436,126.250679,163.011636


In [58]:
# Percentage of missing values
df.isnull().sum().sort_values(ascending=False)

Last_Default_Date                                                                                                                     7873
monthly_income                                                                                                                        1112
annual_income                                                                                                                         1112
DTI_ratio                                                                                                                             1112
Avg_Utilization_Ratio                                                                                                                    1
Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2       1
Total_Ct_Chng_Q4_Q1                                                                                                                      1
Total_Trans_Amt            